In [1]:
import pandas as pd
import gc
from sklearn.model_selection import train_test_split # , StratifiedKFold, KFold, cross_validate, cross_val_predict
from sklearn.metrics import roc_auc_score
#import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt

import catboost as cb
#import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn import tree
from sklearn import ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

import optuna 
# from ydata_profiling import ProfileReport

# Подготовка данных

### Тут создаются удобные CSV файлы для работы алгоритмов в дальнейшем.
### В процессе данные категориальные фичи разбиваются на множество новых фичей через алгоритм One-hot-encoding.

In [4]:
# Объединяю тестовые данные

train_data_0 = pd.concat([
                     pd.read_parquet('train_data_0.pq'), pd.read_parquet('train_data_1.pq'), 
                     pd.read_parquet('train_data_2.pq'), pd.read_parquet('train_data_3.pq'),
                     pd.read_parquet('train_data_4.pq'), pd.read_parquet('train_data_5.pq'),
                     pd.read_parquet('train_data_6.pq'), pd.read_parquet('train_data_7.pq'),
                     pd.read_parquet('train_data_8.pq'),pd.read_parquet('train_data_9.pq'),
                     pd.read_parquet('train_data_10.pq'),pd.read_parquet('train_data_11.pq'),
                     ])

In [4]:
train_data_0 ## NA (пропусков) - нет

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450625,2999999,8,6,5,14,13,1,15,16,2,...,0,0,0,1,1,3,4,1,0,0
2450626,2999999,9,5,3,2,10,15,14,17,2,...,0,0,0,4,1,3,4,1,0,0
2450627,2999999,10,3,16,11,13,14,8,15,5,...,0,0,3,4,1,2,4,1,0,0
2450628,2999999,11,3,6,4,8,1,11,0,5,...,3,3,3,4,1,2,3,1,1,1


In [20]:
train_data_0.describe()

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
count,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,...,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07,2.616272e+07
mean,1.545557e+06,7.047500e+00,9.363500e+00,8.223900e+00,8.365800e+00,8.330800e+00,7.140600e+00,8.362800e+00,9.676600e+00,2.363700e+00,...,2.316100e+00,2.347700e+00,2.381400e+00,3.530300e+00,1.034100e+00,2.693100e+00,3.519100e+00,1.001700e+00,1.492000e-01,2.290000e-01
std,8.679880e+05,5.396700e+00,5.775100e+00,4.713200e+00,5.324500e+00,4.478000e+00,5.152400e+00,4.223300e+00,5.827300e+00,1.310300e+00,...,1.245900e+00,1.225200e+00,1.202000e+00,1.080400e+00,3.049000e-01,4.998000e-01,1.037600e+00,4.880000e-02,3.563000e-01,4.202000e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.949020e+05,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.000000e+00,2.000000e+00,5.000000e+00,5.000000e+00,2.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,1.568512e+06,6.000000e+00,9.000000e+00,9.000000e+00,8.000000e+00,8.000000e+00,7.000000e+00,1.000000e+01,1.000000e+01,2.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,2.303681e+06,1.000000e+01,1.400000e+01,1.200000e+01,1.400000e+01,1.200000e+01,1.200000e+01,1.100000e+01,1.500000e+01,2.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,2.999999e+06,5.800000e+01,1.900000e+01,1.700000e+01,1.700000e+01,1.600000e+01,1.600000e+01,1.500000e+01,1.900000e+01,6.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,6.000000e+00,6.000000e+00,7.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00


In [21]:
train_data_0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26162717 entries, 0 to 2450629
Data columns (total 61 columns):
 #   Column                         Dtype
---  ------                         -----
 0   id                             int64
 1   rn                             int64
 2   pre_since_opened               int64
 3   pre_since_confirmed            int64
 4   pre_pterm                      int64
 5   pre_fterm                      int64
 6   pre_till_pclose                int64
 7   pre_till_fclose                int64
 8   pre_loans_credit_limit         int64
 9   pre_loans_next_pay_summ        int64
 10  pre_loans_outstanding          int64
 11  pre_loans_total_overdue        int64
 12  pre_loans_max_overdue_sum      int64
 13  pre_loans_credit_cost_rate     int64
 14  pre_loans5                     int64
 15  pre_loans530                   int64
 16  pre_loans3060                  int64
 17  pre_loans6090                  int64
 18  pre_loans90                    int64
 19  is_z

### heatmap работает криво даже на части данных и считается долго. 
### но можно конкретно посмотреть корреляцию между двумя признаками по отдельности

In [15]:
train_data_0['pre_over2limit'].corr(train_data_0['pre_util']) # - посмотреть корреляцию конкретных двух признаков

-0.2664241970578944

In [3]:
feature_columns = list(train_data_0.columns.values)
feature_columns.remove("id")
feature_columns.remove("rn")

In [4]:
dummies = pd.get_dummies(train_data_0[feature_columns], columns=feature_columns) # разобью на dummy - переменные (one-hot-encoding)
dummy_features = dummies.columns.values # перекину их в numpy.ndarray

In [5]:
dummies

,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,pre_since_opened_9,...,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,True
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
4,False,False,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450625,False,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,True,False,True,False
2450626,False,False,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
2450627,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
2450628,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,True


In [8]:
train_data_0 = train_data_0.drop(columns=feature_columns)

In [ ]:
train_data_0.memory_usage().sum() # 12 976 707 632 -> 627 905 208

In [ ]:
dummies.memory_usage().sum() # 10,404 gb # 11 171 480 159

In [9]:
ohe_features = pd.concat([train_data_0, dummies], axis=1)

In [10]:
ohe_features

,id,rn,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,...,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1
0,0,1,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
1,0,2,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
2,0,3,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,True
3,0,4,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,True,False,True,False
4,0,5,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450625,2999999,8,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,True,False,True,False
2450626,2999999,9,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,True,False,True,False
2450627,2999999,10,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
2450628,2999999,11,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,True,False,True


In [ ]:
ohe_features.memory_usage().sum() # 11 590 083 631

In [11]:
# уберем уже не нужные df из памяти
del dummies
del train_data_0
gc.collect() 

7

### Сгруппируем полученный DF (ohe_features) по идентификаторам заявки (Id), суммируя каждую из dummy колонок по количеству True значений в ней. 

#### Так как не получается произвести .groupby("id")[dummy_features].sum() сразу по всем 419 колонкам (не хватает оперативной памяти (требуется 82 gb)), я произвожу groupby 3 раза по отдельности по 140 колонкам, сохряняю каждую в память. После склеиваю 

In [ ]:
features_140 = ohe_features.groupby("id")[dummy_features[:140]].sum().reset_index(drop=False)

In [ ]:
features_140.to_csv("features_140.csv", index=None) 

In [ ]:
# 1 блок
del features_140
gc.collect()

In [ ]:
features_140_to_280 = ohe_features.groupby("id")[dummy_features[280:]].sum().reset_index(drop=False)

In [ ]:
features_140_to_280.to_csv("features_140_to_280.csv", index=None) 

In [ ]:
# 2-ой блок
del features_140_to_280
gc.collect()

In [ ]:
features_280_to_end = ohe_features.groupby("id")[dummy_features[280:]].sum().reset_index(drop=False)

In [ ]:
features_280_to_end.to_csv("features_280_to_end.csv", index=None) 

In [ ]:
# 3-ий блок
del features_280_to_end
gc.collect()

### Скелейка блоков

In [ ]:
d1 = pd.read_csv('features_140.csv')
d2 = pd.read_csv('features_140_to_280.csv')
d3 = pd.read_csv('features_280_to_end.csv')

In [ ]:
features = d1.merge(d2, on="id").merge(d3, on="id")

In [ ]:
features.to_csv("features_full.csv", index=None) 

In [ ]:
features.isnull().any().any() # False (выведет True если есть Na) -- Вывод нет Na

### То же преобразование надо сделать с test_data

In [15]:
# просто замените исходный DF train_data_0 на test_data_0
test_data_0 = pd.concat([
                     pd.read_parquet('test_data_0.pq'), pd.read_parquet('test_data_1.pq')])

#### Дальше можно произвести аналитику например через ProfileReport, но не советую его строить так как очень долго строится, отчасти из за тяжелой матрицы корреляций признаков

In [17]:
train_dataset = pd.read_csv('features_full.csv')
train_dataset

,id,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,...,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1
0,0,0,1,1,1,1,2,0,1,0,...,0,0,0,10,0,0,9,1,8,2
1,1,0,0,1,0,0,0,0,1,2,...,0,0,0,14,0,0,13,1,12,2
2,2,1,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,1,2,1,2
3,3,0,3,1,0,2,1,3,0,0,...,0,0,0,15,0,0,10,5,9,6
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2999995,1,2,0,0,2,0,1,0,0,...,0,1,0,11,0,0,9,2,6,5
2999996,2999996,0,1,0,1,1,5,0,0,1,...,0,1,0,13,0,0,8,5,9,4
2999997,2999997,0,0,1,1,3,0,0,0,0,...,0,0,0,10,0,0,9,1,9,1
2999998,2999998,0,0,0,0,1,0,0,1,0,...,0,0,0,5,0,0,3,2,4,1


In [ ]:

from ydata_profiling import ProfileReport
profile1 = ProfileReport(train_dataset, title="Pandas Profiling Report")
profile1

In [18]:
train_dataset.describe()

,id,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,...,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1
count,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,...,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06,3.000000e+06
mean,1.500000e+06,5.206243e-01,4.336140e-01,4.302413e-01,4.039777e-01,4.184067e-01,3.988923e-01,5.076273e-01,4.358403e-01,4.687480e-01,...,1.229100e-02,1.034057e-01,2.850000e-03,8.700294e+00,1.770067e-02,6.100000e-05,7.419765e+00,1.301141e+00,6.724097e+00,1.996808e+00
std,8.660255e+05,9.166651e-01,7.613959e-01,7.628022e-01,7.436818e-01,7.733512e-01,7.487056e-01,8.833271e-01,7.755212e-01,9.608845e-01,...,1.107366e-01,3.998695e-01,6.145903e-02,6.170750e+00,1.691056e-01,8.386277e-03,5.662041e+00,1.365645e+00,5.422938e+00,1.688054e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.499998e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00
50%,1.500000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,1.000000e+00,5.000000e+00,2.000000e+00
75%,2.249999e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.200000e+01,0.000000e+00,0.000000e+00,1.100000e+01,2.000000e+00,1.000000e+01,3.000000e+00
max,2.999999e+06,1.700000e+01,2.900000e+01,3.200000e+01,1.900000e+01,2.400000e+01,2.300000e+01,2.000000e+01,2.600000e+01,1.900000e+01,...,6.000000e+00,1.500000e+01,8.000000e+00,5.800000e+01,1.800000e+01,3.000000e+00,5.000000e+01,1.900000e+01,5.600000e+01,2.300000e+01


# Импорт данных уже подготовленных

In [2]:
train_target = pd.read_csv('train_target.csv') 


In [3]:
train_dataset = pd.read_csv('features_full.csv') 

In [4]:
feature_cols = list(train_dataset.columns.values)
feature_cols.remove("id")
len(feature_cols)

419

In [5]:
test_data = pd.read_csv('test_data_whole_after_f.csv')
feature_cols_test = list(test_data.columns.values)
feature_cols_test.remove("id")
len(feature_cols_test)


425

In [6]:
feature_cols_both = list(set(feature_cols) & set(feature_cols_test)) # оставляю только те фичи которые есть и в треине и в тесте
len(feature_cols_both)

415

In [7]:
X = train_dataset[feature_cols_both]
y = train_target["flag"]

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)# параметр stratify = y на практике только ухудщил показатели

In [9]:
del train_dataset
del train_target
gc.collect()

0

# МОДЕЛЬ 1: Логистическая регрессия 

In [ ]:
scaler_data = StandardScaler()
scaler_data.fit(X_train)

X_scaled = scaler_data.transform(X_train)
val_scaled = scaler_data.transform(X_val)

In [ ]:
clf = LogisticRegression(solver='liblinear',  verbose=True, random_state=42, max_iter = 50)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
roc_auc_score(y_val, clf.predict_proba(X_val)[:,1])

In [ ]:
roc_auc_score(y_val, clf.predict_proba(val_scaled)[:,1])

In [ ]:
test_data = pd.read_csv('test_data_whole_after_f.csv')


In [ ]:
x_col = list(X_train.columns.values)

In [ ]:
score = clf.predict_proba(test_data[x_col])

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score[:, 1]
}) 

submission.to_csv("submission_logreg_2.csv", index=None)  # 0.7154804405 

 ### Предварительный Score 0.7154804405 на предварительных параметрах

In [ ]:
del X_scaled
del val_scaled
del scaler_data
del clf
gc.collect()

# МОДЕЛЬ 2: Дерево решений

In [ ]:
clf = tree.DecisionTreeClassifier()

In [ ]:
scaler_data = StandardScaler()
scaler_data.fit(X_train)

X_scaled = scaler_data.transform(X_train)
val_scaled = scaler_data.transform(X_val)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)############################### stratify=y

In [ ]:
clf = clf.fit(X_train, y_train)

In [ ]:
roc_auc_score(y_val, clf.predict_proba(X_val)[:,1])

In [ ]:
roc_auc_score(y_val, clf.predict_proba(val_scaled)[:,1])

In [ ]:
score = clf.predict_proba(test_data[x_col])

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score[:, 1]
}) 

submission.to_csv("submission_tree.csv", index=None)  

 ### Предварительный Score  0.5272881255676691 - сразу отмету дерево из быстрых решений

# МОДЕЛЬ 3: Случайный лес

In [ ]:
rf = ensemble.RandomForestClassifier(verbose=True, random_state=42)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
roc_auc_score(y_val, rf.predict_proba(X_val)[:,1])

In [ ]:
score = rf.predict_proba(test_data[x_col])

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score[:, 1]
}) 

submission.to_csv("submission_rf_with_no_na.csv", index=None)  # 0.684540263

 ### Предварительный Score  0.684540263 - что даже хуже чем логистическая, стоит опробовать бустинги и стекинг

# Переходим к бустингам

# МОДЕЛЬ 4:  Cat Boost

In [ ]:
cat = cb.CatBoostClassifier(task_type = 'GPU', 
                            depth=10, 
                            iterations = 5000, 
                            random_state = 42, 
                            learning_rate = 0.03,
                            boosting_type='Ordered',
                            #l2_leaf_reg = 3,
                            #bagging_temperature=1,
                            #random_strength=1,
                            early_stopping_rounds = 150,
                            eval_metric = 'AUC'
                            )

In [ ]:
cat.fit(X_train, y_train, eval_set = (X_val, y_val), verbose = True)

In [ ]:
roc_auc_score(y_val, cat.predict_proba(X_val)[:,1])

#0.763 19 33371143584 на дефолт фичах

# 0.763 10 96224055095 на feature_cols_both

# 0.7633454773344337 повторно

#0.7746499455944871

#0.7654337172594504

# 0.7576206497888742


 ### Предварительный Score 0.7495258998

In [ ]:
score = cat.predict_proba(test_data[feature_cols_both])

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score[:, 1]
}) 

submission.to_csv("submission_cat_v5_AUC_with_no_stratify_and_new_param.csv", index=None) # auc прироста не дал, new params тоже

# МОДЕЛЬ 5: XG Boost

In [ ]:
params={
    'device' : 'cuda',
    'verbosity' : 1,
    'alpha': 33,
    'eta' : 0.005, # learning rate
    'max_depth' : 45,
    'gamma' : 2, # lareger - less overfit
    'min_child_weight' : 2, # lareger - less overfit, more conservatrive default 1
    'subsample' : 0.5,
    # 'lambda' : 33,
    'tree_method' : 'hist',
    'objective' : 'binary:logistic',
    'eval_metric' : 'auc',
    #'num_round' : 1000,
    #'early_stopping_rounds' : 150,
    'seed' : 42,
    'n_estimators' : 13000,

}
xgb_model_7 = xgb.XGBClassifier(**params)

In [11]:
xgb_model_7.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = True)

[0]	validation_0-auc:0.69203
[1]	validation_0-auc:0.70222
[2]	validation_0-auc:0.70365
[3]	validation_0-auc:0.70540
[4]	validation_0-auc:0.70643
[5]	validation_0-auc:0.70658
[6]	validation_0-auc:0.70709
[7]	validation_0-auc:0.70689
[8]	validation_0-auc:0.70703
[9]	validation_0-auc:0.70682
[10]	validation_0-auc:0.70758
[11]	validation_0-auc:0.70856
[12]	validation_0-auc:0.70972
[13]	validation_0-auc:0.70948
[14]	validation_0-auc:0.70965
[15]	validation_0-auc:0.71039
[16]	validation_0-auc:0.71065
[17]	validation_0-auc:0.71091
[18]	validation_0-auc:0.71086
[19]	validation_0-auc:0.71082
[20]	validation_0-auc:0.71099
[21]	validation_0-auc:0.71146
[22]	validation_0-auc:0.71191
[23]	validation_0-auc:0.71196
[24]	validation_0-auc:0.71197
[25]	validation_0-auc:0.71196
[26]	validation_0-auc:0.71200
[27]	validation_0-auc:0.71213
[28]	validation_0-auc:0.71277
[29]	validation_0-auc:0.71282
[30]	validation_0-auc:0.71275
[31]	validation_0-auc:0.71287
[32]	validation_0-auc:0.71299
[33]	validation_0-au

XGBClassifier(alpha=33, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eta=0.005, eval_metric='auc',
              feature_types=None, gamma=2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=45,
              max_leaves=None, min_child_weight=2, missing=nan,
              monotone_constraints=None, multi_strategy=None,
              n_estimators=13000, n_jobs=None, ...)

In [13]:
roc_auc_score(y_val, xgb_model_7.predict_proba(X_val)[:,1])

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:54:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


0.7706624507155706

In [14]:
score = xgb_model_7.predict_proba(test_data[feature_cols_both])

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score[:, 1]
}) 

submission.to_csv("submission_xgb_v7_new_full_data_and_params_no_strat.csv", index=None) 

### Предварительный Score: 0.7488144825 - лучший среди моделей без подбора гиперпараметров, после подбор лучшим стал 0.7536345839

# МОДЕЛЬ 6: HistGradientBoosting

In [16]:
hist_model_4 = HistGradientBoostingClassifier (learning_rate = 0.005, 
                                               max_iter = 1000, 
                                               early_stopping = True, 
                                               max_depth = 45, 
                                               validation_fraction = 0.1, #Proportion (or absolute size) of training data to set aside as validation data for early stopping.
                                               verbose= 1,
                                               random_state=42,
                                               #class_weight = 'balanced'
                                               ) #categorical_features = cat_cols_for_hist

In [17]:
hist_model_4.fit(X, y)

Binning 8.964 GB of training data: 6.582 s
Binning 0.996 GB of validation data: 0.387 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.15320, val loss: 0.15320, in 1.463s
[2/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.15310, val loss: 0.15310, in 1.489s
[3/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.15300, val loss: 0.15300, in 1.475s
[4/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.15291, val loss: 0.15290, in 1.457s
[5/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.15281, val loss: 0.15281, in 1.379s
[6/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.15272, val loss: 0.15272, in 1.408s
[7/1000] 1 tree, 31 leaves, max depth = 7, train loss: 0.15262, val loss: 0.15263, in 1.398s
[8/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.15253, val loss: 0.15254, in 1.398s
[9/1000] 1 tree, 31 leaves, max depth = 8, train loss: 0.15244, val loss: 0.15245, in 1.409s
[10/1000] 1 tree, 31 leaves, max depth = 8

HistGradientBoostingClassifier(early_stopping=True, learning_rate=0.005,
                               max_depth=45, max_iter=1000, random_state=42,
                               verbose=1)

In [37]:
score = hist_model_4.predict_proba(test_data[feature_cols_both])

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score[:, 1]
}) 

submission.to_csv("submission_hist_v3.csv", index=None) 

In [18]:
roc_auc_score(y_val, hist_model_4.predict_proba(X_val)[:,1]) # 0.7778393153174453 

0.758361353722427

In [44]:
del hist_model_4,  hist_model_3
gc.collect()

1226

### Предварительный Score: 0.7479819239 - на сайте Яндекс - хороший результат

# МОДЕЛЬ 7: KNN K ближайших соседей

In [54]:
KNN_model_1 = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

In [55]:
KNN_model_1.fit(X, y)

KNeighborsClassifier(n_jobs=-1)

In [56]:
score = KNN_model_1.predict_proba(test_data[feature_cols_both])

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score[:, 1]
}) 

submission.to_csv("submission_knn_v1.csv", index=None) 

# 0.5543549967

In [ ]:
roc_auc_score(y_val, KNN_model_1.predict_proba(X_val)[:,1]) # 0.5543549967

### Предварительный Score: 0.5543549967 - на сайте Яндекс - плохой результат

# Подбор гиперпараметров

In [ ]:

# from optuna.samplers import TPESampler #from optuna.integration import CatBoostPruningCallback

###  OPTUNA Считается долго, очень затратна по помяти, может вылететь с ошибкой в процессе, вообщем грид серч использовал в данном контесте, так как оптуна вылетала спустя N итераций с нехваткой памяти.

In [9]:
# optuna Для CATBOOST
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X,y, test_size=0.25)

    params = {
        "task_type": "GPU",
        "depth": trial.suggest_int("depth", 1, 16, step=1, log=False),
        "iterations": trial.suggest_int("iterations", 1000, 15000, step=1000, log=False),
        "random_state": 42, 
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.101, step = 0.005),
        'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered' , 'Plain']),
        'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1, 50, step = 1)
        # "max_leaves": trial.suggest_int("iterations", 10, 100, step=5, log=False),                    
    }

    cat = cb.CatBoostClassifier(**params)
    cat.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=200)
    score = roc_auc_score(valid_y, cat.predict_proba(valid_x)[:,1])

    return score

In [10]:
pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(pruner=pruner, direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

#print("Best params" + study.best_params)

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-16 03:32:07,651] A new study created in memory with name: no-name-ad40395d-fab2-4ad2-bb50-84786e8e1af0
[I 2024-04-16 03:38:17,491] Trial 0 finished with value: 0.7642295401442047 and parameters: {'depth': 14, 'iterations': 2000, 'learning_rate': 0.091, 'boosting_type': 'Plain', 'l2_leaf_reg': 40.0}. Best is trial 0 with value: 0.7642295401442047.
[I 2024-04-16 03:52:32,380] Trial 1 finished with value: 0.7609171928908729 and parameters: {'depth': 3, 'iterations': 12000, 'learning_rate': 0.031, 'boosting_type': 'Ordered', 'l2_leaf_reg': 42.0}. Best is trial 0 with value: 0.7642295401442047.
[I 2024-04-16 03:59:18,906] Trial 2 finished with value: 0.7628777331367224 and parameters: {'depth': 5, 'iterations': 9000, 'learning_rate': 0.08600000000000001, 'boosting_type': 'Ordered', 'l2_leaf_reg': 32.0}. Best is trial 0 with value: 0.7642295401442047.
[I 2024-04-16 03:59:55,871] Trial 3 finished with value: 0.7585133403344733 and parameters: {'depth': 5, 'iterations': 1000, 'learn

: 

In [11]:
def objective(trial):
    dtrain=xgb.DMatrix(X, label = y)
    param = {
        #"n_estimators" : trial.suggest_int("n_estimators", 100, 5000),
        'random_state': trial.suggest_categorical('random_state', [42]),
        "device" : "cuda",
        #"verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        # "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 0.001, 1.0),
        "alpha": trial.suggest_float("alpha", 0.001, 1.0)
    }

    
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "test-auc") # убирает заведомо худшие параметры
    history = xgb.cv(param, dtrain, num_boost_round=100, callbacks=[pruning_callback],early_stopping_rounds=50)
    mean_auc = history["test-auc-mean"].values[-1]
    return mean_auc

In [12]:
pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(pruner=pruner, direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

#print("Best params" + study.best_params)

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-16 14:41:18,481] A new study created in memory with name: no-name-01065568-3e0f-443b-ab9f-cd066f30403c
[I 2024-04-16 14:53:30,435] Trial 0 finished with value: 0.7557915288473375 and parameters: {'random_state': 42, 'lambda': 0.05941557275906488, 'alpha': 0.008167556561827896}. Best is trial 0 with value: 0.7557915288473375.
[I 2024-04-16 15:54:15,892] Trial 1 finished with value: 0.7552965489012927 and parameters: {'random_state': 42, 'lambda': 0.023947573980379198, 'alpha': 0.0159580981350516}. Best is trial 0 with value: 0.7557915288473375.
[W 2024-04-16 15:55:57,519] Trial 2 failed with parameters: {'random_state': 42, 'lambda': 0.02032580295049346, 'alpha': 0.3385184822791768} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:

KeyboardInterrupt: 

### Submission

In [ ]:
score = cat.predict_proba(test_data)

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score[:, 1]
}) 

submission.to_csv("submission_cat_iterations = 1000, random_state = 42, learning_rate = 0.05.csv", index=None) 

In [ ]:
submission

# Интерпретация и SHAP значения

In [14]:
import shap 

#### Возьму модель одну из самых результативных моделей XGB в качестве примера

In [11]:
params={
    'device' : 'cuda',
    'verbosity' : 1,
    'alpha': 33,
    'eta' : 0.005, # learning rate
    'max_depth' : 30, 
    'gamma' : 2, # lareger - less overfit
    'min_child_weight' : 2, # lareger - less overfit, more conservatrive default 1
    'subsample' : 0.5,
    # 'lambda' : 33,
    'tree_method' : 'hist',
    'objective' : 'binary:logistic',
    'eval_metric' : 'auc',
    #'num_round' : 1000,
    #'early_stopping_rounds' : 150,
    'seed' : 42,
    'n_estimators' : 5000,

}
xgb_model_7 = xgb.XGBClassifier(**params)

In [12]:
xgb_model_7.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = True)

[0]	validation_0-auc:0.69203
[1]	validation_0-auc:0.70222
[2]	validation_0-auc:0.70365
[3]	validation_0-auc:0.70540
[4]	validation_0-auc:0.70643
[5]	validation_0-auc:0.70658
[6]	validation_0-auc:0.70709
[7]	validation_0-auc:0.70689
[8]	validation_0-auc:0.70703
[9]	validation_0-auc:0.70682
[10]	validation_0-auc:0.70758
[11]	validation_0-auc:0.70856
[12]	validation_0-auc:0.70972
[13]	validation_0-auc:0.70948
[14]	validation_0-auc:0.70965
[15]	validation_0-auc:0.71039
[16]	validation_0-auc:0.71065
[17]	validation_0-auc:0.71091
[18]	validation_0-auc:0.71086
[19]	validation_0-auc:0.71082
[20]	validation_0-auc:0.71099
[21]	validation_0-auc:0.71146
[22]	validation_0-auc:0.71191
[23]	validation_0-auc:0.71196
[24]	validation_0-auc:0.71197
[25]	validation_0-auc:0.71196
[26]	validation_0-auc:0.71200
[27]	validation_0-auc:0.71212
[28]	validation_0-auc:0.71277
[29]	validation_0-auc:0.71282
[30]	validation_0-auc:0.71275
[31]	validation_0-auc:0.71287
[32]	validation_0-auc:0.71299
[33]	validation_0-au

XGBClassifier(alpha=33, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eta=0.005, eval_metric='auc',
              feature_types=None, gamma=2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=30,
              max_leaves=None, min_child_weight=2, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=5000,
              n_jobs=None, ...)

In [15]:
explainer = shap.TreeExplainer(xgb_model_7)

In [16]:
shap_values = explainer(X)
shap_values1 = explainer.shap_values(X) 

In [ ]:
type(shap_values)

In [ ]:
type(shap_values1)

In [ ]:
shap_values1.shape

In [ ]:
shap.dependence_plot("rank(0)", shap_values1, X)

In [ ]:
shap.plots.initjs()

In [ ]:
shap.plots.bar(shap_values, max_display=99)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=99)